# Setup

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

## Physical Models of target and interceptor

In [ ]:
from models.missile import PhysicalMissleModel
import models.physics as physics
import numpy as np

interceptor_speed = physics.mach_to_ms(4.0)  # Speed of the interceptor in m/s
target_speed = physics.mach_to_ms(3.0)  # Speed of the target in m/s
target = PhysicalMissleModel(velocity=np.array([0, target_speed, 0.0]), max_acc=100 * 9.81, pos=np.array([0.0, -10_000, 15_000.0]))
interceptor = PhysicalMissleModel(velocity=np.array([0.0, 0.0, interceptor_speed]), max_acc=100 * 9.81, pos=np.array([0.0, 0.0, 100.0]))

## Environment & Gym

In [ ]:
from gym.environment import MissileEnv, MissileEnvSettings
from pilots.random_evasion_pilot import RandomEvasionPilot

# target behavior
target_pilot = RandomEvasionPilot(aggression=0.2, trajectory_maintainance=0.01)
target_pilot = None

settings = MissileEnvSettings()
settings.time_step = 0.01    # Time step for the simulation
settings.realtime = False    # Runs faster than real-time
settings.time_limit = 50.0  # Time-limit for the episode
env = MissileEnv(settings=settings, target=target, interceptor=interceptor, target_pilot=target_pilot)

# Proportional Guidance

In [ ]:
from pilots.proportional_nav_pilot import PlanarProportionalNavPilot, SpaceProportionalNavPilot
from visualization.matplot_viz import MatplotVisualizer
import time


# setup pilots
proportional_nav_pilot = PlanarProportionalNavPilot(max_acc=100*9.81, max_speed=interceptor_speed, n=3)


done = False
obs, _ = env.reset()

viz = MatplotVisualizer()
viz.set_episode_data(env.current_episode)

last_time = time.time()
max_render_fps = 10.0

while not done:
    # get un-normalized observations for the interceptor (environment outputs normalized observations for RL agent)
    obs = env.get_interceptor_observations(settings.time_step).pack()

    # pilot the interceptor using the proportional navigation algorithm
    action = proportional_nav_pilot.step(obs, settings.time_step)
    obs, reward, done, _, _ = env.step(action)  # Take a step in the environment
    
# viz.playback(env.sim_time, speed=5.0)
viz.render(env.sim_time)

# Reinforcement Learning Agents

## Soft Actor-Critic

In [ ]:
from stable_baselines3 import SAC
from tqdm import tqdm

model = SAC("MlpPolicy", env, verbose=1, tensorboard_log="./.logs/sac", device="cuda")
with tqdm(total=100_000, desc="Training Progress", unit="steps") as pbar:
    model.learn(total_timesteps=100_000, progress_bar=pbar)

viz.set_episode_data(env.current_episode)
viz.render(env.sim_time)

In [ ]:
def get_best_distance(episode):
    # get the best distance between the interceptor and the target
    return min([state.distance for time, state in env.current_episode.get_interceptor("Agent").states.all.items()])

print(f"Best distance: {get_best_distance(env.current_episode)} m")

In [ ]:
# Train until desired results
complete = False
while not complete:
    with tqdm(total=10_000, desc="Training Progress", unit="steps") as pbar:
        model.learn(total_timesteps=10_000, progress_bar=pbar)

    eval_distances = []
    for i in range(10):
        done = False
        while not done:
            obs = env.get_interceptor_observations(settings.time_step).pack()
            action, _states = model.predict(obs, deterministic=True)
            obs, reward, done, _, _ = env.step(action)

        best_distance = min([state.distance for time, state in env.current_episode.get_interceptor("Agent").states.all.items()])
        eval_distances.append(best_distance)
    
    mean_distance = np.mean(eval_distances)
    print(f"Mean distance: {mean_distance:.2f} m")
    
    if mean_distance < 100.0:
        complete = True
        print(f"Training complete! Mean distance: {mean_distance:.2f} m")

In [ ]:
obs, _ = env.reset()
done = False
while not done:
    action, _states = model.predict(obs, deterministic=True)
    obs, rewards, done, _, _ = env.step(action)

# Results

## Evaluate

In [ ]:
best_distances = []

for i in range(10):
    obs, _ = env.reset()
    done = False
    while not done:
        action, _states = model.predict(obs, deterministic=True)
        obs, rewards, done, _, _ = env.step(action)
        
    distances = [state.distance for time, state in env.current_episode.get_interceptor("Agent").states.all.items()]
    best_distances.append(min(distances))

best_distance = min(best_distances)
print(f"Best distance: {best_distance:.2f} m")

## Save and Playback

In [ ]:
for id, interceptor_state in env.current_episode.interceptors.items():
    distances = [state.distance for time , state in interceptor_state.states.all.items()]
    best_distance = min(distances)
    print (f"Interceptor {id} best distance: {best_distance:.2f} m")

viz.set_episode_data(env.current_episode)
viz.save_playback("output.gif", env.sim_time, 5.0, 5)

## Plot Distance over time

In [ ]:
# get all distances from the interceptor to the target
distances = {time: state.distance for time, state in env.current_episode.get_interceptor("Agent").states.all.items()}

import matplotlib.pyplot as plt
import numpy as np

times = list(distances.keys())
distance_values = list(distances.values())

plt.figure(figsize=(10, 6))
plt.plot(times, distance_values, label="Distance over Time")

# Find the minimum value and its corresponding time
min_distance = min(distance_values)
min_time = times[distance_values.index(min_distance)]

# Add a label for the minimum value
plt.scatter(min_time, min_distance, color='red', label=f"Min Distance: {min_distance:.2f} m")
plt.text(min_time, min_distance, f"({min_time:.2f}, {min_distance:.2f})", color='red', fontsize=10)

plt.xlabel("Time (s)")
plt.ylabel("Distance (m)")
plt.title("Interceptor to Target Distance Over Time")
plt.legend()
plt.grid(True)
plt.show()